## Create Tool by Function

In [3]:
from configparser import ConfigParser
from langchain.llms import OpenAI
import os

config = ConfigParser()
config.read("../config.ini")
os.environ["OPENAI_API_KEY"] = config["KEY"]["openai_key"]

In [4]:
from autogen import ConversableAgent

In [5]:
assistant = ConversableAgent(
        name="Assistant",
        system_message="You are a helpful chatbot. You can help answer question based on input document. Return 'TERMINATE' when task is done",
        llm_config={"config_list": [{"model":"gpt-3.5-turbo", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

In [9]:
user_proxy = ConversableAgent(
        name="User",
        llm_config=False,
        is_termination_msg= lambda msg: msg["content"] is not None and "TERMINATE" in msg["content"],
        human_input_mode="NEVER"
)

In [10]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [18]:
def retriever(pdf_url:str , query:str):
        docs = PyPDFLoader(pdf_url, extract_images=True).load()
        vector_db = Chroma.from_documents(
                docs,
                embedding=OpenAIEmbeddings()
        )
        retrieved_docs = vector_db.similarity_search(query, k=2)
        return "\n\n".join([doc.page_content for doc in retrieved_docs])

In [19]:
from autogen import register_function

register_function(
        retriever,
        caller=assistant,
        executor=user_proxy,
        name="retriever",
        description="A retriever who can return  the relevant information for  a query based on pdf documents"
)

The return type of the function 'retriever' is not annotated. Although annotating it is optional, the function should return either a string, a subclass of 'pydantic.BaseModel'.
/Users/chessman/miniforge3/envs/PyJupyter/lib/python3.10/site-packages/autogen/agentchat/conversable_agent.py:2573: UserWarning: Function 'retriever' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
/Users/chessman/miniforge3/envs/PyJupyter/lib/python3.10/site-packages/autogen/agentchat/conversable_agent.py:2492: UserWarning: Function 'retriever' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


In [20]:
chat_result = user_proxy.initiate_chat(
        assistant,
        message="Based on https://arxiv.org/pdf/1706.03762 paper, can you tell me what is encoder"
)

User (to Assistant):

Based on https://arxiv.org/pdf/1706.03762 paper, can you tell me what is encoder

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_68XhiC5XmdSzr2iWioWq04PS): retriever *****
Arguments: 
{"pdf_url":"https://arxiv.org/pdf/1706.03762","query":"encoder"}
**************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION retriever...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_68XhiC5XmdSzr2iWioWq04PS) *****
1 Introduction
Recurrent neural networks, long short-term memory [ 13] and gated recurrent [ 7] neural networks
in particular, have been firmly established as state of the art approaches in sequence modeling and
transduction problems such as language modeling and machine translation [ 35,2,5]. Nu